In [1]:
import pandas as pd
from thefuzz import process
import re

# Cargar el archivo Excel (ajusta el nombre y la ruta del archivo)
df_consolidado = pd.read_excel("Consolidado-Nacional-20232-publico-3WEB.xlsx", engine="openpyxl")

# Mostrar las primeras filas del DataFrame
df_consolidado.head

<bound method NDFrame.head of                         Nombre Comercial            Actividad  \
0                                FAMA LA  ALIMENTOS Y BEBIDAS   
1                          QUINTA ELOISA       INTERMEDIACIÓN   
2      VILLA VERDE RECEPCIONES Y EVENTOS       INTERMEDIACIÓN   
3                          HOTEL CORDERO          ALOJAMIENTO   
4                  HOTEL PUERTO BALLESTA          ALOJAMIENTO   
...                                  ...                  ...   
23211                         GLORY TOUR           OPERACIÓN    
23212                   OPENINGFRONTIERS       INTERMEDIACIÓN   
23213                         HAPPY ROCK  ALIMENTOS Y BEBIDAS   
23214                    WILD  WEST CAFE  ALIMENTOS Y BEBIDAS   
23215                           PAUSA LA  ALIMENTOS Y BEBIDAS   

                        Clasificiación               Categoría    Provincia  \
0                            CAFETERÍA            (1) Una taza        AZUAY   
1      SALA DE RECEPCIONES Y BA

In [2]:
df_filtrado = df_consolidado[
    (df_consolidado["Actividad"] == "ALIMENTOS Y BEBIDAS") & 
    (df_consolidado["Provincia"].isin(["MANABÍ", "MANABI"])) &
    (df_consolidado["Cantón"] =="MANTA" ) &
    (df_consolidado["Parroquia"] =="MANTA" )
    
]

df_filtrado.head


<bound method NDFrame.head of                                     Nombre Comercial            Actividad  \
102                                        PALMEIRAS  ALIMENTOS Y BEBIDAS   
1110                                          BREZZA  ALIMENTOS Y BEBIDAS   
1169                         EMPANADAS DE DANITO LAS  ALIMENTOS Y BEBIDAS   
1505   RESTAURANTE CERVECERIA ARTESANAL LATITUD CERO  ALIMENTOS Y BEBIDAS   
1506   RESTAURANTE CERVECERIA ARTESANAL LATITUD CERO  ALIMENTOS Y BEBIDAS   
...                                              ...                  ...   
20497                                      PIZZA HUT  ALIMENTOS Y BEBIDAS   
21050                             LUPULO BEER GARDEN  ALIMENTOS Y BEBIDAS   
23102                                      PIPCOFFEE  ALIMENTOS Y BEBIDAS   
23104                                      PIPCOFFEE  ALIMENTOS Y BEBIDAS   
23108                                      PIPCOFFEE  ALIMENTOS Y BEBIDAS   

      Clasificiación             Categoría Pr

In [3]:
df_columns_drop=df_filtrado.drop(columns=["Referencia de Dirección", "Teléfono Principal", "Teléfono Secundario",
"Correo Electrónico","Dirección Web","Provincia", "Cantón", "Parroquia", "Tipo de Parroquia", "Actividad"
])

df_columns_drop.head

<bound method NDFrame.head of                                     Nombre Comercial Clasificiación  \
102                                        PALMEIRAS    RESTAURANTE   
1110                                          BREZZA            BAR   
1169                         EMPANADAS DE DANITO LAS      CAFETERÍA   
1505   RESTAURANTE CERVECERIA ARTESANAL LATITUD CERO    RESTAURANTE   
1506   RESTAURANTE CERVECERIA ARTESANAL LATITUD CERO            BAR   
...                                              ...            ...   
20497                                      PIZZA HUT    RESTAURANTE   
21050                             LUPULO BEER GARDEN    RESTAURANTE   
23102                                      PIPCOFFEE      CAFETERÍA   
23104                                      PIPCOFFEE      CAFETERÍA   
23108                                      PIPCOFFEE      CAFETERÍA   

                  Categoría                                   Dirección  
102      (3) Tres tenedores  AV. 29 Y CIRCU

In [4]:
df_csv_interpreter = pd.read_csv("interpreter (1).csv",sep="\t")

df_csv_interpreter.head()

,name,cuisine,addr:street,@lat,@lon
0,Parque del Marisco,NaN,NaN,-0.950525,-80.712285
1,NaN,NaN,NaN,5.009049,-73.540304
2,Camilo y Mateo,NaN,NaN,-0.985980,-80.848416
3,NaN,NaN,NaN,29.587505,32.723589
4,Cevicheria Umiña,seafood;fish_and_chips,Calle 27 ave 30,-0.946359,-80.739129


In [5]:
# Convertir ambas columnas a mayúsculas
df_columns_drop["Nombre Comercial"]=df_columns_drop["Nombre Comercial"].str.upper()
df_csv_interpreter["name"]=df_csv_interpreter["name"].str.upper()
df_csv_interpreter.to_csv("archivo_salida3.csv", sep=",", index=False, encoding="utf-8")


# Crear una nueva columna para la latitud
df_columns_drop["latitud"] = None 
# Crear una nueva columna para la longitud
df_columns_drop["longitud"] = None 
# Umbral de similitud (0-100, donde 100 es coincidencia exacta)
umbral_similitud = 80  

# Iterar sobre los nombres en df_consolidado
for i, row in df_columns_drop.iterrows():
    nombre = row["Nombre Comercial"]
    
    # Buscar la mejor coincidencia en df_csv_interpreter["name"]
    resultado = process.extractOne(nombre, df_csv_interpreter["name"])
    mejor_coincidencia = resultado[0]  # El nombre de la mejor coincidencia
    score = resultado[1]  # El puntaje de la coincidencia
    
    # Si la coincidencia es mayor al umbral, asignar el valor de @lat
    if score >= umbral_similitud:
        df_columns_drop.at[i, "latitud"] = df_csv_interpreter.loc[df_csv_interpreter["name"] == mejor_coincidencia, "@lat"].values[0]
        df_columns_drop.at[i, "longitud"] = df_csv_interpreter.loc[df_csv_interpreter["name"] == mejor_coincidencia, "@lon"].values[0]

print(df_columns_drop)

                                    Nombre Comercial Clasificiación  \
102                                        PALMEIRAS    RESTAURANTE   
1110                                          BREZZA            BAR   
1169                         EMPANADAS DE DANITO LAS      CAFETERÍA   
1505   RESTAURANTE CERVECERIA ARTESANAL LATITUD CERO    RESTAURANTE   
1506   RESTAURANTE CERVECERIA ARTESANAL LATITUD CERO            BAR   
...                                              ...            ...   
20497                                      PIZZA HUT    RESTAURANTE   
21050                             LUPULO BEER GARDEN    RESTAURANTE   
23102                                      PIPCOFFEE      CAFETERÍA   
23104                                      PIPCOFFEE      CAFETERÍA   
23108                                      PIPCOFFEE      CAFETERÍA   

                  Categoría                                   Dirección  \
102      (3) Tres tenedores  AV. 29 Y CIRCUNVALACION Y AV. FLAVIO REYES 

In [6]:
# Eliminar filas donde la columna 'latitud' tenga valores nulos
df_columns_drop = df_columns_drop.dropna(subset=["latitud"])
# Función para extraer el número y clasificarlo
def extraer_valores(categoria):
    match = re.match(r"\((\d+)\) (.+)", categoria)  # Extrae el número y la descripción
    if match:
        cantidad = int(match.group(1))  # Extrae el número dentro de los paréntesis
        descripcion = match.group(2).lower()  # Convierte la descripción a minúsculas para evitar errores
        
        # Asignar valores según la descripción
        if "copa" in descripcion:
            return cantidad, 0, 0  # numero_copas, numero_tazas, numero_cubiertos
        elif "taza" in descripcion:
            return 0, cantidad, 0
        elif "tenedor" in descripcion:
            return 0, 0, cantidad
    return 0, 0, 0  # Si no coincide con ninguna categoría

# Aplicar la función a cada fila y crear nuevas columnas
df_columns_drop[["numero_copas", "numero_taza", "numero_cubiertos"]] = df_columns_drop["Categoría"].apply(lambda x: pd.Series(extraer_valores(x)))



C:\Users\bryar.PITO-SAN\AppData\Local\Temp\ipykernel_10796\549093207.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_columns_drop[["numero_copas", "numero_taza", "numero_cubiertos"]] = df_columns_drop["Categoría"].apply(lambda x: pd.Series(extraer_valores(x)))
C:\Users\bryar.PITO-SAN\AppData\Local\Temp\ipykernel_10796\549093207.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_columns_drop[["numero_copas", "numero_taza", "numero_cubiertos"]] = df_columns_drop["Categoría"].apply(lambda x: pd.Se

In [7]:
# Diccionario para mapear valores
mapa_clasificacion = {
    "CAFETERÍA": "Cafetería",
    "BAR": "Bar",
    "RESTAURANTE": "Restaurante",
    "DISCOTECA": "Discoteca",
    "ESTABLECIMIENTO MÓVIL": "Establecimiento móvil",
    "PLAZA DE COMIDA": "Plaza de comida",
    "SERVICIO CATERING": "Servicio Catering"
}

# Crear la nueva columna 'tipo' mapeando los valores
df_columns_drop["tipo"] = df_columns_drop["Clasificiación"].map(mapa_clasificacion)

df_columns_drop.head()

C:\Users\bryar.PITO-SAN\AppData\Local\Temp\ipykernel_10796\1050107937.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_columns_drop["tipo"] = df_columns_drop["Clasificiación"].map(mapa_clasificacion)


,Nombre Comercial,Clasificiación,Categoría,Dirección,latitud,longitud,numero_copas,numero_taza,numero_cubiertos,tipo
102,PALMEIRAS,RESTAURANTE,(3) Tres tenedores,AV. 29 Y CIRCUNVALACION Y AV. FLAVIO REYES,-0.945845,-80.738022,0,0,3,Restaurante
1169,EMPANADAS DE DANITO LAS,CAFETERÍA,(1) Una taza,AV. 24 ENTRE CALLE 17 Y 20,-0.946315,-80.734032,0,1,0,Cafetería
1505,RESTAURANTE CERVECERIA ARTESANAL LATITUD CERO,RESTAURANTE,(1) Un tenedor,"UNIVERSITARIA 2,CALLE U8",-0.975613,-80.839115,0,0,1,Restaurante
1506,RESTAURANTE CERVECERIA ARTESANAL LATITUD CERO,BAR,(3) Tres copas,"UNIVERSITARIA 2,CALLE U8",-0.975613,-80.839115,3,0,0,Bar
3248,CREPERIA EL MEDIODIA,CAFETERÍA,(1) Una taza,AV. MALECON MALL DEL PACIFICO 20-23 EN EL INT...,-0.942371,-80.730538,0,1,0,Cafetería


In [8]:
df_columns_drop.rename(columns={'Nombre Comercial': 'nombre', 'Dirección': 'direccion'}, inplace=True)
df_columns_drop.drop(columns=['Clasificiación', 'Categoría'], inplace=True)

df_columns_drop.head()

C:\Users\bryar.PITO-SAN\AppData\Local\Temp\ipykernel_10796\3485817410.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_columns_drop.rename(columns={'Nombre Comercial': 'nombre', 'Dirección': 'direccion'}, inplace=True)
C:\Users\bryar.PITO-SAN\AppData\Local\Temp\ipykernel_10796\3485817410.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_columns_drop.drop(columns=['Clasificiación', 'Categoría'], inplace=True)


,nombre,direccion,latitud,longitud,numero_copas,numero_taza,numero_cubiertos,tipo
102,PALMEIRAS,AV. 29 Y CIRCUNVALACION Y AV. FLAVIO REYES,-0.945845,-80.738022,0,0,3,Restaurante
1169,EMPANADAS DE DANITO LAS,AV. 24 ENTRE CALLE 17 Y 20,-0.946315,-80.734032,0,1,0,Cafetería
1505,RESTAURANTE CERVECERIA ARTESANAL LATITUD CERO,"UNIVERSITARIA 2,CALLE U8",-0.975613,-80.839115,0,0,1,Restaurante
1506,RESTAURANTE CERVECERIA ARTESANAL LATITUD CERO,"UNIVERSITARIA 2,CALLE U8",-0.975613,-80.839115,3,0,0,Bar
3248,CREPERIA EL MEDIODIA,AV. MALECON MALL DEL PACIFICO 20-23 EN EL INT...,-0.942371,-80.730538,0,1,0,Cafetería


In [9]:
df_columns_drop = df_columns_drop.drop_duplicates(subset='nombre', keep='first')

df_columns_drop.to_csv("archivo_salida.csv", sep=",", index=False, encoding="utf-8")